In [10]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Read and standarize the dataset
df = pd.read_csv('data/selected_data.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns.values)

In [17]:
def hotellingsT2(components):
    x = components.transpose()    
    mean_vec = np.mean(components, axis=0)
    cov = (components - mean_vec).T.dot((components - mean_vec)) / (components.shape[0]-1)
    w = np.linalg.solve(cov, x)
    t2 = (x * w).sum(axis=0)
    return t2

In [19]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2).fit(X)

components = pca.fit_transform(X)

len(hotellingsT2(components))

10014

In [33]:
hald_text = """Y       X1      X2      X3      X4
    78.5    7       26      6       60
    74.3    1       29      15      52
    104.3   11      56      8       20
    87.6    11      31      8       47
    95.9    7       52      6       33
    109.2   11      55      9       22
    102.7   3       71      17      6
    72.5    1       31      22      44
    93.1    2       54      18      22
    115.9   21      47      4       26
    83.8    1       40      23      34
    113.3   11      66      9       12
    109.4   10      68      8       12
    """

hald = np.loadtxt(hald_text.splitlines(), skiprows=1)
ingredients = hald[:, 1:]

pca = PCA().fit_transform(ingredients)
hotellingsT2(pca)

array([5.68034084, 3.07583704, 6.00023279, 2.61976323, 3.36813945,
       0.56679667, 3.48184073, 3.9793979 , 2.60858624, 7.48175633,
       4.18302223, 2.23271872, 2.72156782])